<a href="https://colab.research.google.com/github/machine-perception-robotics-group/MPRGDeepLearningLectureNotebook/blob/master/12_gan/03_Deep_Convolutional_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Convolutional Generative Adversarial Networks (DC-GAN)
## 目的
DC-GANによって画像の生成をして動作を理解する．

## 必要なモジュールのインポート
Pytorchで学習するときに必要となるモジュールをインポートします．

In [ ]:
import os
import zipfile
import urllib.request
import numpy as np
from PIL import Image
%matplotlib inline
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms

## ネットワークの構築
オリジナルのGANは，全結合層とReLUのみで構築される非常にシンプルなネットワークでした．<br>
シンプルな反面で，学習が不安定になったり，高解像な画像生成とは言い難いものでした．

Deep Convolutional GAN (DC-GAN)は，名前の通り畳み込み処理を用いたネットワークの構築することによって，オリジナルのGANより綺麗な画像を生成することを可能としました．
他にもDC-GANは，以下に示す点でもオリジナルのGANと異なります．
* Batch Normalizationを利用する．（Discriminatorの1層目と最終層は除く）
* Discriminatorの活性化関数は，Leaky ReLUを使用する．
* Generatorの最終層はtanh関数を使用する．（それ以外の層は全てReLU）
* Deconvolutionの使用，PoolingではなくConvolutionによるDownsampling

これらを考慮してDiscriminatorとGeneratorのネットワークを構築します．<br>
GeneratorとDiscriminator共に重みパラメータは平均が0，偏差が0.02の正規分布で，バイアスは0で初期化します．

In [ ]:
class Generator(nn.Module):
    def __init__(self, latent_dim=100, out_ch=3, img_size=32):
        super(Generator, self).__init__()
        n_layer = int(np.log2(img_size)) - 2
        self.model = nn.Sequential()
        
        features = 512
        self.model.add_module('Deconv_first', nn.ConvTranspose2d(latent_dim, features, 2, 2, bias=True))
        self.model.add_module('BN_first', nn.BatchNorm2d(features))
        self.model.add_module('Act_first', nn.ReLU(inplace=True))
  
        in_features = features
        for i in range(n_layer):
            out_features = in_features // 2
            self.model.add_module('Deconv%d' % i, nn.ConvTranspose2d(in_features, out_features, 4, 2, 1, bias=True))
            self.model.add_module('BN%d' % i, nn.BatchNorm2d(out_features))
            self.model.add_module('Act%d' % i, nn.ReLU(inplace=True))
            in_features = out_features
        
        self.model.add_module('Deconv_last', nn.ConvTranspose2d(in_features, out_ch, 2, 2, bias=True))
        self.model.add_module('Act_last', nn.Tanh())
        self.weight_init()

    def weight_init(self):
        for m in self.model:
          if isinstance(m, nn.ConvTranspose2d) or isinstance(m, nn.Conv2d):
            m.weight.data.normal_(0.0, 0.02)
            m.bias.data.zero_()

    def forward(self, z):
        return self.model(z)
    
class Discriminator(nn.Module):
    def __init__(self, in_ch=3, img_size=32):
        super(Discriminator, self).__init__()
        n_layer = int(np.log2(img_size)) - 2
        self.model = nn.Sequential()
        
        features = 32
        self.model.add_module('Conv_first', nn.Conv2d(in_ch, features, 4, 2, 1, bias=True))
        self.model.add_module('Act_first', nn.LeakyReLU(negative_slope=0.2))
  
        in_features = features
        for i in range(n_layer):
            out_features = in_features * 2
            self.model.add_module('Conv%d' % i, nn.Conv2d(in_features, out_features, 4, 2, 1, bias=True))
            self.model.add_module('BN%d' % i, nn.BatchNorm2d(out_features))
            self.model.add_module('Act%d' % i, nn.LeakyReLU(negative_slope=0.2))
            in_features = out_features
        
        self.model.add_module('Conv_last', nn.Conv2d(in_features, 1, 2, 1, bias=True))
        self.weight_init()
  
    def weight_init(self):
        for m in self.model:
          if isinstance(m, nn.ConvTranspose2d) or isinstance(m, nn.Conv2d):
            m.weight.data.normal_(0.0, 0.02)
            m.bias.data.zero_()

    def forward(self, x):
        return self.model(x).view(-1)

## データセットと最適化関数
データセットにはMNISTを使用します．
最適化関数はAdam optimizer使用し，学習率$2\times 10^4$，betaの値を$0.5, 0.999$として学習します．

In [ ]:
class MNIST_Loader(Dataset):
  def __init__(self, train=True, download=True, transform=None):
    self.transform = transform
    self.url = 'https://www.dropbox.com/s/hc7ukm7vzst5e40/MNIST.zip?dl=1'
    
    if download:
      self._download()
    
    dname = os.path.join(self.url.rpartition('/')[2][:-9], 'processed')
    if train:
      datapath = os.path.join(dname, 'training.pt')
    else:
      datapath = os.path.join(dname, 'test.pt')
  
    self.data = torch.load(datapath)

  def __len__(self):
    return len(self.data[0])

  def _download(self):
    filename = self.url.rpartition('/')[2][:-5]
    urllib.request.urlretrieve(self.url, filename)

    with zipfile.ZipFile(filename) as existing_zip:
      existing_zip.extractall()
    os.remove(filename)

  def __getitem__(self, i):
    image = transforms.ToPILImage()(self.data[0][i])
    label = self.data[1][i]

    if self.transform:
      image = self.transform(image)
    return image, label

In [ ]:
transform_training = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor()])
try:
  mnist_data = datasets.MNIST(root='./data', train=True, transform=transform_training, download=True)
except:
  print('Warning: Switch the original implementation because official MNIST data did not download (probably the official server has down).')
  mnist_data = MNIST_Loader(train=True, download=True, transform=transform_training)
print('Done!')
training_data = DataLoader(mnist_data, batch_size=128, shuffle=True)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
latent_dim = 100
G = Generator(latent_dim=latent_dim, out_ch=1, img_size=32).to(device)
D = Discriminator(in_ch=1, img_size=32).to(device)
opt_g = optim.Adam(G.parameters(), lr=0.0002, betas=(0.5, 0.999))
opt_d = optim.Adam(D.parameters(), lr=0.0002, betas=(0.5, 0.999))
print('=' * 50)
print(G)
print('=' * 50)
print(D)
print('=' * 50)

## GANの学習
学習のプロセスは，オリジナルのGANと同じです．

In [ ]:
n_epoch = 20
n_critic = 1
criterion = nn.BCEWithLogitsLoss()
print(device)
for epoch in range(1, n_epoch+1):
    Tensor = torch.cuda.FloatTensor
    G.train()
    D.train()
    for idx, (real_x, _) in enumerate(training_data):
        real_x = real_x.to(device)
        batch = real_x.size(0)
        flag_real = Tensor(batch).fill_(1.0)
        flag_fake = Tensor(batch).fill_(0.0)
        
        for _ in range(n_critic):
            D.zero_grad()
            z = torch.randn(batch, latent_dim, 1, 1).to(device)
            fake_x = G(z)
            out_real = D(real_x)
            out_fake = D(fake_x.detach())
            loss_real = criterion(out_real, flag_real)
            loss_fake = criterion(out_fake, flag_fake)
            dis_loss = loss_real + loss_fake
            dis_loss.backward()
            opt_d.step()
            
        G.zero_grad()
        z = torch.randn(batch, latent_dim, 1, 1).to(device)
        fake_x = G(z)
        out_gen = D(fake_x)
        gen_loss = criterion(out_gen, flag_real)
        gen_loss.backward()
        opt_g.step()
        
        if idx % 100 == 0:
            print('Training epoch: {} [{}/{} ({:.0f}%)] | D loss: {:.6f} | G loss: {:.6f} |'\
                  .format(epoch, idx * len(real_x), len(training_data.dataset),
                  100. * idx / len(training_data), dis_loss.item(), gen_loss.item()))

In [ ]:
z = torch.randn(100, 100, 1, 1).to(device)
G.eval()
with torch.no_grad():
  test_img = G(z)
_test_img = (test_img * 256.).clamp(min=0., max=255.).squeeze().data.cpu().numpy()

fig = plt.figure(figsize=(10, 10))
for i, im in enumerate(_test_img):
    ax = fig.add_subplot(10, 10, i+1, xticks=[], yticks=[])
    ax.imshow(im, 'gray')

## 課題
1. データセットをMNISTからCIFAR-10に変更して学習してみましょう．
2. 2つの潜在変数の間を補間するように画像生成した場合，どのような画像が生成されるか確認してください．

# 参考文献
[1] Alec Radford, Luke Metz and Soumith Chintala, Unsupervised Representation Learning with Deep Convolutional Generative Adversarial Networks, ICLR, 2016.